# TP1 Science des donnees - Chakirou ALABANI (alac_3501) - Amadou Selle NDIAYE (ndia2909)

In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cosine,mahalanobis

: 

In [ ]:
data = pd.read_csv('data.csv')
print(data.shape)
data.head()

In [ ]:
data_labels = pd.read_csv('labels.csv')
print(data_labels.shape)
data_labels.head()

In [ ]:
print(data_labels['Class'][0])

# Merge

In [ ]:
df = pd.merge(data,data_labels, on='Unnamed: 0',how='inner')
df.head()

# Pre-traitement

In [ ]:
#Suppression de 'Unnamed: 0'
df= df.drop(columns=['Unnamed: 0'],axis=1)
df.head()

In [ ]:
#Suppression de toutes les colonnes avec plus de 75 % de valeurs nulles
colonnes_a_supprimer=[]

for i in df.columns:
    s=0
    for j in range(df.shape[0]):
        if df[i][j] == 0:
            s+=1
    if (s/df.shape[0]) >= 0.75:
        colonnes_a_supprimer.append(i)

#print(colonnes_a_supprimer)
len(colonnes_a_supprimer)

In [9]:
df_filtree = df.drop(columns=colonnes_a_supprimer,axis=1)

In [ ]:
df_filtree.shape

In [ ]:
#Representation des classes
df_filtree['Class'].value_counts()

# Methode 1: Calcul de distances

## 1.1 Calcul des centres de classes

In [12]:
#centre des classes
centre_des_classe= {}

liste_classe = df_filtree['Class'].unique()

for classe in liste_classe:
    df_classe_i = df_filtree[df_filtree['Class']==classe]
    centre_df_classe_i = df_classe_i.drop(columns='Class',axis=1).mean()
    #print(centre_df_classe_i)
    centre_des_classe[classe]= centre_df_classe_i


## 1.2 Distance Euclidienne

            ### Distance intra classe

In [ ]:

distance_intra_classe_euclid={}

for classe  in liste_classe:
    
    dfclasse=df_filtree[df_filtree['Class']==classe]
    dfclasse_without_target=dfclasse.drop(columns='Class',axis=1)

    
    dist = np.linalg.norm(dfclasse_without_target- centre_des_classe[classe],axis=1)
    
    distance_intra_classe_euclid[classe]= max(dist)


for key, value in distance_intra_classe_euclid.items():
    print(f"La distance intra-classe entre {key} est {value:.2f}")

            ### Distance inter classe

In [ ]:
distance_inter_classe_euclid={}

for i  in range(0,len(liste_classe)):

    dfclasse_i=df_filtree[df_filtree['Class']==liste_classe[i]]
    dfclasse_i_without_target=dfclasse_i.drop(columns='Class',axis=1)

    for j in range(i+1,len(liste_classe)):
        
        min_list=[]

        
        dfclasse_j=df_filtree[df_filtree['Class']==liste_classe[j]]
        dfclasse_j_without_target=dfclasse_j.drop(columns='Class',axis=1)

        
        dist_i_j = np.linalg.norm(dfclasse_i_without_target - centre_des_classe[liste_classe[j]],axis=1)
        min_list.append(min(dist_i_j))

        dist_j_i = np.linalg.norm(dfclasse_j_without_target - centre_des_classe[liste_classe[i]],axis=1)
        min_list.append(min(dist_j_i))

        distance_inter_classe_euclid[f"{liste_classe[i]} - {liste_classe[j]}"]= min(min_list)



for key, value in distance_inter_classe_euclid.items():
    print(f"La distance inter-classe entre {key} est {value:.2f}")


            ### OverLap

In [ ]:
overlap_euclid={}

for i  in range(0,len(liste_classe)):
    classe_i =liste_classe[i]
    for j in range(i+1,len(liste_classe)):
        
        classe_j =liste_classe[j]

        distance_inter_i_j= distance_inter_classe_euclid[f"{classe_i} - {classe_j}"]

        overlap_i_j = (distance_intra_classe_euclid[classe_i] + distance_intra_classe_euclid[classe_j])/ (2 * distance_inter_i_j)
        overlap_euclid[f"{classe_i} - {classe_j}"]=overlap_i_j


for key, value in overlap_euclid.items():
    print(f"L'overlap de {key} est {value:.2f}")

    if value < 1:
        print(f"les classes {key} sont bien separees.")
    else:
        print(f"les classes {key} ne sont pas bien separees.")


## 1.3 Distance Cosinus

            ### Distance intra classe

In [ ]:
distance_intra_classe_cosinus={}

for classe  in liste_classe:
    list_dist=[]
    
    dfclasse=df_filtree[df_filtree['Class']==classe]
    dfclasse_without_target=dfclasse.drop(columns='Class',axis=1)

    for i in range(dfclasse_without_target.shape[0]):
        dist = cosine(dfclasse_without_target.iloc[i,:],centre_des_classe[classe])
        list_dist.append(dist)

    distance_intra_classe_cosinus[classe]= max(list_dist)


for key, value in distance_intra_classe_cosinus.items():
    print(f"La distance intra-classe entre {key} est {value:.4f}")

            ### Distance inter classe

In [ ]:
distance_inter_classe_coinus={}

for i  in range(0,len(liste_classe)):

    dfclasse_i=df_filtree[df_filtree['Class']==liste_classe[i]]
    dfclasse_i_without_target=dfclasse_i.drop(columns='Class',axis=1)

    for j in range(i+1,len(liste_classe)):
        
        
        min_list=[]

        dfclasse_j=df_filtree[df_filtree['Class']==liste_classe[j]]
        dfclasse_j_without_target=dfclasse_j.drop(columns='Class',axis=1)

        listx=[]
        for x in range(dfclasse_i_without_target.shape[0]):
            dist_i_j = cosine(dfclasse_i_without_target.iloc[x,:], centre_des_classe[liste_classe[j]])
            listx.append(dist_i_j)

        min_list.append(min(listx))
        
        listz=[]
        for z in range(dfclasse_j_without_target.shape[0]):
            dist_j_i = cosine(dfclasse_j_without_target.iloc[z,:], centre_des_classe[liste_classe[i]])
            listz.append(dist_j_i)

        min_list.append(min(listz))
        
        distance_inter_classe_coinus[f"{liste_classe[i]} - {liste_classe[j]}"]= min(min_list)



for key, value in distance_inter_classe_coinus.items():
    print(f"La distance inter-classe entre {key} est {value:.4f}")


            ### Overlap

In [ ]:
overlap_cosinus={}

for i  in range(0,len(liste_classe)):
    classe_i =liste_classe[i]
    for j in range(i+1,len(liste_classe)):
        
        classe_j =liste_classe[j]

        distance_inter_i_j= distance_inter_classe_coinus[f"{classe_i} - {classe_j}"]

        overlap_i_j = (distance_intra_classe_cosinus[classe_i] + distance_intra_classe_cosinus[classe_j])/ (2 * distance_inter_i_j)
        overlap_cosinus[f"{classe_i} - {classe_j}"]=overlap_i_j


for key, value in overlap_cosinus.items():
    print(f"L'overlap de {key} est {value:.2f}")

    if value < 1:
        print(f"les classes {key} sont bien separees.")
    else:
        print(f"les classes {key} ne sont pas bien separees.")


## 1.4 Distance Mahalanobis

In [19]:
df_mahalanobis_selection=df_filtree.iloc[:,13700:13850]
df_mahalanobis_selection['Class']=df_filtree['Class']

In [52]:
#centre des classes
centre_des_classe_mahalanobis= {}


for classe in liste_classe:
    df_classe_i = df_mahalanobis_selection[df_mahalanobis_selection['Class']==classe]
    centre_df_classe_i = df_classe_i.drop(columns='Class',axis=1).mean()
    
    centre_des_classe_mahalanobis[classe]= centre_df_classe_i


            ### Distance intra classe

In [ ]:
distance_intra_classe_mahalanobis={}

for classe  in liste_classe:
    list_dist=[]
    
    dfclasse=df_mahalanobis_selection[df_mahalanobis_selection['Class']==classe]
    dfclasse_without_target=dfclasse.drop(columns='Class',axis=1).values

    cov= np.cov(dfclasse_without_target.T,ddof=1)

    for i in range(dfclasse_without_target.shape[0]):
        dist = mahalanobis(dfclasse_without_target[i,:],centre_des_classe_mahalanobis[classe],np.linalg.pinv(cov))
        list_dist.append(dist)

    distance_intra_classe_mahalanobis[classe]= max(list_dist)


for key, value in distance_intra_classe_mahalanobis.items():
    print(f"La distance intra-classe entre {key} est {value:.2f}")

            ### Distance inter classe

In [ ]:
distance_inter_classe_mahalanobis={}

for i  in range(0,len(liste_classe)):

   dfclasse_i = df_mahalanobis_selection[df_mahalanobis_selection['Class']==liste_classe[i]]
   dfclasse_without_target_i = dfclasse.drop(columns='Class',axis=1)

   list_dist =[]

   for j in range(i+1,len(liste_classe)):
      dfclasse_j = df_mahalanobis_selection[df_mahalanobis_selection['Class']==liste_classe[j]]
      dfclasse_without_target_j = dfclasse.drop(columns='Class',axis=1)
   
   
      listx=[]
      for x in range(dfclasse_without_target_i.shape[0]):
         dist_i_j = mahalanobis(dfclasse_without_target_i.iloc[x,:],centre_des_classe_mahalanobis[liste_classe[j]].values,np.linalg.pinv(np.cov(dfclasse_without_target_j.T)))
         listx.append(dist_i_j)

      list_dist.append(min(listx))

      listz=[]
      for z in range(dfclasse_without_target_j.shape[0]):
         dist_j_i = mahalanobis(dfclasse_without_target_j.iloc[z,:],centre_des_classe_mahalanobis[liste_classe[i]].values,np.linalg.pinv(np.cov(dfclasse_without_target_i.T)))
         listz.append(dist_j_i)
      
      list_dist.append(min(listz))

      distance_inter_classe_mahalanobis[f"{liste_classe[i]} - {liste_classe[j]}"]= min(list_dist)

for key, value in distance_inter_classe_mahalanobis.items():
   print(f"La distance inter-classe entre {key} est {value:.2f}")


            ### Overlap

In [ ]:
overlap_mahalanobis={}

for i  in range(0,len(liste_classe)):
    classe_i =liste_classe[i]
    for j in range(i+1,len(liste_classe)):
        
        classe_j =liste_classe[j]

        distance_inter_i_j= distance_inter_classe_mahalanobis[f"{classe_i} - {classe_j}"]

        overlap_i_j = (distance_intra_classe_mahalanobis[classe_i] + distance_intra_classe_mahalanobis[classe_j])/ (2 * distance_inter_i_j)
        overlap_mahalanobis[f"{classe_i} - {classe_j}"]=overlap_i_j


for key, value in overlap_mahalanobis.items():
    print(f"L'overlap de {key} est {value:.2f}")

    if value < 1:
        print(f"les classes {key} sont bien separees.")
    else:
        print(f"les classes {key} ne sont pas bien separees.")


# Notes

- Je vais merge les deux datasets pour avoir la correspondance de chaque espece a sa classe
- Apres je vais faire un pre-traitement vu qu'il y'a beaucoup de colonnes qui n'ont pas assez d'informations (Beaucoup de 0). La colonne Unamed:0 pourra etre enleve aussi.
- Ensuite je pourrai proceder au calcul de distance
- distance euclidienne la fonction nous permet l'operation par vecteur
- distance cosinus on doit parcourir le df par ligne et faire l'addition
- Lors du calcul de la matrice inv on a utilise pinv() au lieu de lieu de inv() par ce que les matrices n'etaient pas carres